In [1]:
import requests
import config
import json
import pandas as pd
#from requests.auth import HTTPBasicAuth

username = 'KseniaBuk'
api_username = config.username
password = config.password 

url = f'https://api.ravelry.com/people/{username}/favorites/list.json'
print(url)

https://api.ravelry.com/people/KseniaBuk/favorites/list.json


In [2]:
payload = {
    'page': str(1),
    'page_size': 25
}
response = requests.get(url, params=payload, auth=(api_username, password))

In [3]:
if response.status_code == 200:
    data = response.json()

pages = data['paginator']['page_count']
favorites = data.get('favorites', [])
favorites_trunc = []
p_id = []

for f in favorites:
    fn = {
        'name' : f['favorited']['name'], 
        'created_at' : f['created_at'], 
        'id' : f['favorited']['id'],
        'permalink' : f['favorited']['permalink'],
        'page' : 1
    }
    favorites_trunc.append(fn)
    p_id.append(f['favorited']['id'])

In [4]:
print(pages)

11


In [5]:
for i in range(2, pages  + 1):
    payload = {
        'page': str(i),
        'page_size': 25
    }
    response = requests.get(url, params=payload, auth=(api_username, password))
    data = response.json()
    favorites = data.get('favorites', [])
    for f in favorites:
        fn = {
            'name' : f['favorited']['name'], 
            'created_at' : f['created_at'], 
            'id' : f['favorited']['id'],
            'permalink' : f['favorited']['permalink'],
            'page' : i
        }
        favorites_trunc.append(fn)
        p_id.append(f['favorited']['id'])

In [6]:
favorites_df = pd.json_normalize(favorites_trunc)
favorites_df.sort_values('created_at', ascending = False).head()

,name,created_at,id,permalink,page
0,Dream Vest,2025/09/11 00:48:03 -0400,7391346,dream-vest-2,1
1,Obsidian Slipover,2025/09/11 00:47:09 -0400,7433044,obsidian-slipover,1
2,Twist Loop Collar,2025/09/08 23:43:48 -0400,1275995,twist-loop-collar,1
3,Clematis Sweater,2025/09/08 22:00:25 -0400,1340146,clematis-sweater-2,1
4,Eun Sweater Light,2025/09/08 21:57:47 -0400,7382501,eun-sweater-light,1


In [7]:
patterns = []
for i in p_id: 
    url = f'https://api.ravelry.com/patterns/{i}.json'
    response = requests.get(url, auth=(api_username, password))
    if response.status_code == 200:
        data = response.json()
        patterns.append(data)

In [41]:
patterns_trunc = []
for p in patterns:
    pn = {
        'p_id' : p['pattern']['id'], 
        'difficulty_average' : p['pattern']['difficulty_average'],
        'difficulty_count' : p['pattern']['difficulty_count'],
        'gauge' : p['pattern']['gauge'],
        'gauge_divisor' : p['pattern']['gauge_divisor'],
        'gauge_pattern' : p['pattern']['gauge_pattern'],
        'rating_average' : p['pattern']['rating_average'],
        'rating_count' : p['pattern']['rating_count'],
        'row_gauge' : p['pattern']['row_gauge'],
        'url' : p['pattern']['url'],
        'yardage' : p['pattern']['yardage'],
        'yardage_max' : p['pattern']['yardage_max'],
        'yarn_weight_description' : p['pattern']['yarn_weight_description'],
        'yardage_description' : p['pattern']['yardage_description'],
        'knit_gauge' : p['pattern'].get('yarn_weight', {}).get('knit_gauge'),
        'name' : p['pattern'].get('yarn_weight', {}).get('name'),
        'ply' : p['pattern'].get('yarn_weight', {}).get('ply'),
        'wpi' : p['pattern'].get('yarn_weight', {}).get('wpi'),
        'pattern_category' : p['pattern']['pattern_categories'][0]['name'],
        'pattern_parent' : p['pattern']['pattern_categories'][0]['parent']['name']
        
    }
    patterns_trunc.append(pn)

In [42]:
patterns_df = pd.json_normalize(patterns_trunc)
patterns_df.head()

,p_id,difficulty_average,difficulty_count,gauge,gauge_divisor,gauge_pattern,rating_average,rating_count,row_gauge,url,yardage,yardage_max,yarn_weight_description,yardage_description,knit_gauge,name,ply,wpi,pattern_category,pattern_parent
0,7391346,3.250000,16.0,17.0,4.0,,4.400000,15.0,24.0,https://festnetz.shop/products/trad-wife-dream...,NaN,NaN,Worsted (9 wpi),yards,20,Worsted,10,9,Vest,Clothing
1,7433044,6.000000,3.0,19.0,4.0,stockinette,4.666667,3.0,28.0,https://sofiascotfordknitwear.com/products/obs...,656.0,875.0,Worsted (9 wpi),656 - 875 yards,20,Worsted,10,9,Vest,Clothing
2,1275995,3.333333,9.0,17.0,4.0,in pattern,4.777778,9.0,24.0,,689.0,919.0,Aran (8 wpi),689 - 919 yards,18,Aran,10,8,Vest,Clothing
3,1340146,3.720721,111.0,18.0,4.0,stockinette stitch,4.610619,113.0,26.0,,875.0,1914.0,DK (11 wpi),875 - 1914 yards,22,DK,8,11,Pullover,Sweater
4,7382501,3.066667,30.0,21.0,4.0,21 sts x 28 rounds on 4 mm needles = 10 x 10 c...,4.781250,32.0,28.0,https://www.novemberknits.com,NaN,NaN,DK (11 wpi),yards,22,DK,8,11,Pullover,Sweater


In [43]:
favorites_df.to_excel('favorites.xlsx')
patterns_df.to_excel('patterns.xlsx')